# The trial of web scraping with Mediacloud API

In [23]:
import requests
import csv
import os, sys, getopt, datetime
from pathlib import Path

In [7]:
# read API_KEY file
def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    # feel free to modify this function if you are storing the API Key differently
    with open(filepath, 'r') as f:
        return f.read().replace('\n','')
MY_KEY = read_api_key("mediacloud/mediacloud_key.txt")
MY_KEY

'fac5e9dec4b476c273fd33730040e488b3143b3d132e55d1de65a5a592f4a51d'

In [28]:
def write_json_to_csv(json_list, attribute_name_list, outputfile_name):
    """
        args:
            json_list(JsonList): such as stories, media
            attribute_name_list(list): attribute_name_list == column labels
        returns:
            na
    """
    # check if file exist or not, if exist, then rename old file
    
    filepath = ''
    old_name = outputfile_name
    # check if outf containes the directory info
    pos = outputfile_name.rfind('/')
    if pos != -1:
        filepath = outputfile_name[: pos + 1]
        old_name = outputfile_name[pos + 1: ]
    dt = str(datetime.datetime.now())
    new_outf = Path(outputfile_name)
    if new_outf.is_file():
        # if the file exist, rename it
        newname = old_name[0: len(old_name) - 4]+dt+'.csv'
        os.rename(outputfile_name, filepath + newname)
    # use 'x' to create and write the file
    with open(new_outf, 'w') as csvfile:
        print("open")
        cwriter = csv.DictWriter(csvfile, attribute_name_list, extrasaction='ignore')
        cwriter.writeheader()
        cwriter.writerows(json_list)

In [29]:
#Create a CSV file with all media sources.
media = []
start = 0
rows  = 100
while True:
    params = { 'start': start, 'rows': rows, 'key': MY_KEY }
    print("start:{} rows:{}".format( start, rows))
    r = requests.get( 'https://api.mediacloud.org/api/v2/media/list', 
                     params = params, headers = { 'Accept': 'application/json'} )
    data = r.json()

    if len(data) == 0:
        break

    start += rows
    media.extend( data )
    if start >= 200: break

media_fieldnames = [
    u'media_id',
    u'url',
    u'name'
]

write_json_to_csv(media, media_fieldnames, 'tmp/media.csv')

start:0 rows:100
start:100 rows:100
open


# Grab stories by querying stories_public/list

In [30]:
start = 0
rows  = 100
stories_list = []
while True:
    params = {
        'last_processed_stories_id': start,
        'rows': rows,
        'q': 'text:cryptocurrency',
        'fq': 'publish_date:[2019-01-01T00:00:00Z TO 2019-03-01T00:00:00Z]',
        'key': MY_KEY
    }

    print("Fetching {} stories starting from {}".format( rows, start))
    r = requests.get( 'https://api.mediacloud.org/api/v2/stories_public/list/', 
                     params = params, headers = { 'Accept': 'application/json'} )
    stories = r.json()

    if len(stories) == 0:
        break

    start = stories[-1]['processed_stories_id']
    stories_list.extend(stories)
    if start >=300: break

story_list_fieldnames = [
        u'ap_syndicated',
        u'collect_date',
        u'feeds',
        u'guid',
        u'language',
        u'media_id',
        u'media_name',
        u'media_url',
        u'processed_stories_id',
        u'publish_date',
        u'stories_id',
        u'story_tags',
        u'title',
        u'url',
        u'word_count'
        ]
write_json_to_csv(stories_list, story_list_fieldnames, 'tmp/story_list.csv')

Fetching 100 stories starting from 0
open
